In [1]:
import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import string
import os
import sys
import math

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


## Levenshtein Distance Algorithm

In [2]:
def levenshteinDistance(str1, str2): # M*N
  m = len(str1)
  n = len(str2)
  d = [[i] for i in range(1, m + 1)]   # d matrix rows
  d.insert(0, list(range(0, n + 1)))   # d matrix columns
  for j in range(1, n + 1): # A 
    for i in range(1, m + 1): # B
      if str1[i - 1] == str2[j - 1]: # C     # Python (string) is 0-based
        substitutionCost = 0
      else: # D
        substitutionCost = 1
      d[i].insert(j, min(d[i - 1][j] + 1,
                          d[i][j - 1] + 1,
                          d[i - 1][j - 1] + substitutionCost))

In [3]:
def generate_random_string(length):
    return ''.join(random.choice(string.ascii_lowercase) for _ in range(length))

## Frequency Count Method

In [4]:
X_y = []
x1 = []
x2 =[]
y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(levenshteinDistance)

  # create 2 random strings with diferents lenght
  string1 = generate_random_string(n)
  string2 = generate_random_string(n*2)

  lp_wrapper(string1, string2)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  x1.append(n)
  x2.append(n*2)
  y.append(sum(hits))

X_reshaped = np.column_stack((x1, x2))
y_np = np.array(y)


In [5]:
resultados_com_menor_loss = []
repeat = 5
registros = []

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=["log", "square", "cube"],
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=["log", "square", "cube"],
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=["log", "square", "cube"],
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout



[ Info: Started!
0.0%┣                                             ┫ 0/600 [00:00<-1:-14, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           2.647e+08  1.594e+01  y = x₁                                        3           1.368e+08  3.301e-01  y = (x₁ * x₁)                                 5           4.990e+07  5.041e-01  y = ((x₁ + x₁) * x₁)                          6           2.542e+06  2.977e+00  y = ((square(-2.5224) * x₁) * x₀)             8           1.140e+06  4.010e-01  y = ((square(square(log(x₀))) + x₁) * x₁)     10          9.552e+05  

In [6]:
registros_ = registros

In [7]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

print(lista_melhor_valor[4])

C:\Users\rafae\AppData\Local\Temp\ipykernel_7224\7133900.py:5: RuntimeWarning: invalid value encountered in scalar multiply
  w = (loss * score)/complexity


3.5255728*x1**2
